In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.layers import Input,LSTM,Dense
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle

In [5]:
#parameters 
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'fra.txt'

In [6]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [7]:
for line in lines [:min(num_samples,len(lines)-1)]:
    input_text, target_text, _ = line.split('\t') # to split the input text and  the target a
    target_text =  '\t' + target_text + '\n' 
    input_texts.append(input_text.lower())
    target_texts.append(target_text.lower())
    for char in input_text.lower():
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text.lower():
        if char not in target_characters:
            target_characters.add(char)

In [8]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
pickle.dump({'input_characters':input_characters,'target_characters':target_characters,
             'max_input_length':max_encoder_seq_length,'max_target_length':max_decoder_seq_length,
             'num_en_chars':num_encoder_tokens,'num_dec_chars':num_decoder_tokens},open("training_data.pkl","wb"))

In [13]:
def bagofcharacters(input_texts,target_texts):
  #inintialize encoder , decoder input and target data.
  en_in_data=[] ; dec_in_data=[] ; dec_tr_data=[]
  #padding variable with first character as 1 as rest all 0.
  pad_en=[1]+[0]*(len(input_characters)-1)
  pad_dec=[0]*(len(target_characters)) ; pad_dec[2]=1
  #countvectorizer for one hot encoding as we want to tokenize character so
  #anlyzer is true and None the stopwords action.
  cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char')
  for i,(input_t,target_t) in enumerate(zip(input_texts,target_texts)):
    #fit the input characters into the CountVectorizer function
    cv_inp= cv.fit(input_characters)
    
    #transform the input text from the help of CountVectorizer fit.
    #it character present than put 1 and 0 otherwise.
    en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())
    cv_tar= cv.fit(target_characters)		
    dec_in_data.append(cv_tar.transform(list(target_t)).toarray().tolist())
    #decoder target will be one timestep ahead because it will not consider 
    #the first character i.e. '\t'.
    dec_tr_data.append(cv_tar.transform(list(target_t)[1:]).toarray().tolist())
    
    #add padding variable if the length of the input or target text is smaller
    #than their respective maximum input or target length. 
    if len(input_t) < max_encoder_seq_length:
      for _ in range(max_encoder_seq_length-len(input_t)):
        en_in_data[i].append(pad_en)
    if len(target_t) < max_decoder_seq_length:
      for _ in range(max_decoder_seq_length-len(target_t)):
        dec_in_data[i].append(pad_dec)
    if (len(target_t)-1) < max_decoder_seq_length:
      for _ in range(max_decoder_seq_length-len(target_t)+1):
        dec_tr_data[i].append(pad_dec)
  
  #convert list to numpy array with data type float32
  en_in_data=np.array(en_in_data,dtype="float32")
  dec_in_data=np.array(dec_in_data,dtype="float32")
  dec_tr_data=np.array(dec_tr_data,dtype="float32")

  return en_in_data,dec_in_data,dec_tr_data

In [14]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state = True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation= 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics =['accuracy'])

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 47)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 65)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 311296      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  329728      input_4[0][0]                    
                                                                 lstm_2[0][1]          

In [15]:
encoder_input_data,decoder_input_data,decoder_target_data = bagofcharacters(input_texts,target_texts)

In [16]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

Epoch 1/100
125/125 [==============================] - 53s 423ms/step - loss: 1.2775 - accuracy: 0.7209 - val_loss: 1.0733 - val_accuracy: 0.6993
Epoch 2/100
125/125 [==============================] - 54s 431ms/step - loss: 0.8817 - accuracy: 0.7551 - val_loss: 0.9012 - val_accuracy: 0.7495
Epoch 3/100
125/125 [==============================] - 52s 415ms/step - loss: 0.7969 - accuracy: 0.7855 - val_loss: 0.8288 - val_accuracy: 0.7665
Epoch 4/100
125/125 [==============================] - 52s 413ms/step - loss: 0.6751 - accuracy: 0.8077 - val_loss: 0.7353 - val_accuracy: 0.7869
Epoch 5/100
125/125 [==============================] - 51s 409ms/step - loss: 0.6192 - accuracy: 0.8207 - val_loss: 0.6901 - val_accuracy: 0.7996
Epoch 6/100
125/125 [==============================] - 53s 424ms/step - loss: 0.5850 - accuracy: 0.8299 - val_loss: 0.6580 - val_accuracy: 0.8066
Epoch 7/100
125/125 [==============================] - 52s 417ms/step - loss: 0.5581 - accuracy: 0.8365 - val_loss: 0.6354 -

Epoch 57/100
125/125 [==============================] - 51s 409ms/step - loss: 0.1466 - accuracy: 0.9552 - val_loss: 0.4858 - val_accuracy: 0.8766
Epoch 58/100
125/125 [==============================] - 50s 402ms/step - loss: 0.1442 - accuracy: 0.9558 - val_loss: 0.4893 - val_accuracy: 0.8764
Epoch 59/100
125/125 [==============================] - 50s 403ms/step - loss: 0.1399 - accuracy: 0.9568 - val_loss: 0.4908 - val_accuracy: 0.8766
Epoch 60/100
125/125 [==============================] - 52s 418ms/step - loss: 0.1353 - accuracy: 0.9587 - val_loss: 0.4969 - val_accuracy: 0.8760
Epoch 61/100
125/125 [==============================] - 50s 402ms/step - loss: 0.1319 - accuracy: 0.9594 - val_loss: 0.4963 - val_accuracy: 0.8769
Epoch 62/100
125/125 [==============================] - 50s 401ms/step - loss: 0.1292 - accuracy: 0.9600 - val_loss: 0.5055 - val_accuracy: 0.8748
Epoch 63/100
125/125 [==============================] - 51s 406ms/step - loss: 0.1255 - accuracy: 0.9614 - val_loss: 0

In [17]:
model.save("fra_eng.h5")